In [30]:
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv('/content/diabetes.csv')

In [32]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [33]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [34]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [37]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [38]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6763 - loss: 0.6390 - val_accuracy: 0.7208 - val_loss: 0.6016
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7206 - loss: 0.5887 - val_accuracy: 0.7662 - val_loss: 0.5637
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7507 - loss: 0.5604 - val_accuracy: 0.7727 - val_loss: 0.5338
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7625 - loss: 0.5417 - val_accuracy: 0.7987 - val_loss: 0.5119
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7711 - loss: 0.5179 - val_accuracy: 0.7987 - val_loss: 0.4966
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7737 - loss: 0.5101 - val_accuracy: 0.7987 - val_loss: 0.4870
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7782 - loss: 0.4882 - val_accuracy: 0.7922 - val_loss: 0.4788
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7633 - loss: 0.4821 - val_accuracy: 0.7987 - val_lo

In [40]:
#1. how to select appropriate optimizer
#2. no. of nodes in layer
#3. how to select no. of layers
#4. all in one model

In [41]:
pip install -U keras-tuner

In [42]:
import kerastuner as kt

In [43]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values = ['rmsprop','adam','adadelta','sgd'])

  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [44]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5)

Reloading Tuner from ./untitled_project/tuner0.json


In [45]:
tuner.search(X_train,y_train, epochs=5,validation_data=(X_test,y_test))

In [46]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [47]:
tuner.get_best_models(num_models=1)[0].summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7868 - loss: 0.4548 - val_accuracy: 0.7987 - val_loss: 0.4631
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7798 - loss: 0.4537 - val_accuracy: 0.7987 - val_loss: 0.4619
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7891 - loss: 0.4566 - val_accuracy: 0.8052 - val_loss: 0.4614
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7725 - loss: 0.4560 - val_accuracy: 0.8052 - val_loss: 0.4606
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8011 - loss: 0.4272 - val_accuracy: 0.8052 - val_loss: 0.4592
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7712 - loss: 0.4648 - val_accuracy: 0.8052 - val_loss: 0.4579
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7883 - loss: 0.4313 - val_accuracy: 0.7987 - val_loss: 0.4587
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7864 - loss: 0.4467 - val_accuracy: 0.80

In [49]:
def build_model0(hp):

  model = Sequential()

  units = hp.Int('units',8,128, step=2)
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [50]:
tuner0 = kt.RandomSearch(build_model0,
                        objective = 'val_accuracy',
                        max_trials=3,
                        directory='project',
                        project_name='superman')

Reloading Tuner from project/superman/tuner0.json


In [51]:
tuner0.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [52]:
tuner0.get_best_hyperparameters()[0].values

{'units': 102}

In [53]:
model = tuner0.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.6206 - loss: 0.6396 - val_accuracy: 0.7727 - val_loss: 0.5583
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7519 - loss: 0.5610 - val_accuracy: 0.7922 - val_loss: 0.5182
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7634 - loss: 0.5271 - val_accuracy: 0.7857 - val_loss: 0.4939
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7549 - loss: 0.5122 - val_accuracy: 0.7792 - val_loss: 0.4823
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8126 - loss: 0.4633 - val_accuracy: 0.7727 - val_loss: 0.4736
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7852 - loss: 0.4627 - val_accuracy: 0.7857 - val_loss: 0.4708
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7697 - loss: 0.4859 - val_accuracy: 0.7987 - val_loss: 0.4670
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7721 - loss: 0.4621 - val_accuracy: 0.

In [ ]:
def build_model1(hp):

  model = Sequential()

  model.add(Dense(102,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=20)):
    model.add(Dense(102,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
tuner1 = kt.RandomSearch(build_model1,
                        objective = 'val_accuracy',
                        max_trials=3,
                        directory='project',
                        project_name='superman')

Reloading Tuner from project/superman/tuner0.json


In [ ]:
tuner1.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner1.get_best_hyperparameters()[0].values

{'units': 102}

In [ ]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8044 - loss: 0.4118 - val_accuracy: 0.8117 - val_loss: 0.4645
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8230 - loss: 0.3846 - val_accuracy: 0.8117 - val_loss: 0.4621
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8362 - loss: 0.3899 - val_accuracy: 0.8117 - val_loss: 0.4585
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8239 - loss: 0.3813 - val_accuracy: 0.8117 - val_loss: 0.4610
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8277 - loss: 0.3736 - val_accuracy: 0.8117 - val_loss: 0.4580
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8190 - loss: 0.3861 - val_accuracy: 0.8052 - val_loss: 0.4598
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8066 - loss: 0.4053 - val_accuracy: 0.7987 - val_loss: 0.4602
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8210 - loss: 0.4030 - val_accuracy: 0.8117 - 